In [1]:
'''
Parameters to customize for every new page
or put in a list before running loop

1. pagename
2. forked_repo_url
'''

'\nParameters to customize for every new page\nor put in a list before running loop\n\n1. pagename\n2. forked_repo_url\n'

In [2]:
#the page to migrate
pagename = 'Cookie_Search'

forked_repo_url = 'https://github.com/rdrsadhu/cookie-search-activity.git'

In [3]:
# Request is an easy-to-read HTTP library in Python
# See: http://docs.python-requests.org/en/master/
import requests

In [4]:
# os helps to run a process as if it was ran from Terminal
import os

In [5]:
#folder to store all related file and use as git repo
command = 'mkdir '+pagename
os.system(command)

0

In [6]:
# changing directory

path = pagename
os.chdir(path)

In [7]:
#Initializing git repository
os.system('git init')

0

In [8]:
command = 'git remote add origin '+forked_repo_url

os.system(command)

0

In [9]:
os.system('git pull origin master')

0

In [10]:
# using mediawiki API to get the wikitext
# see https://www.mediawiki.org/wiki/API:Main_page
url = 'https://wiki.sugarlabs.org/api.php?action=parse&page=Activities/'+pagename+'&prop=wikitext&format=json'

In [11]:
# GET request to url, reutrned value is stored in data
data = requests.get(url) 

In [12]:
if data.status_code == 200:
    
    # storing result in json form
    json_obj = data.json()
    
    # fetching wikitext from json_obj
    wikitext = json_obj['parse']['wikitext']['*']
    
    # storing wikitext in a file to convert using pandoc
    try:
        #file_location = pagename+'/wikitoconvert'
        file_location = 'wikitoconvert'
        file = open(file_location,'w')
        file.write(wikitext)
        print('Wikitext Successfully written to file.')
    finally:
        file.close()

else:
    print('Sorry! Received status_code of GET request = {}'
          .format(data.status_code))

Wikitext Successfully written to file.


In [13]:
# using pandoc to convert from mediawiki markup to markdown

#command = 'pandoc '+pagename+'/wikitoconvert -f mediawiki -t markdown -s -o '+pagename+'/README.md'
command = 'pandoc wikitoconvert -f mediawiki -t markdown -s -o README.md'
os.system(command)

0

In [14]:
# url to get the images used in wiki-page
imageurl = 'https://wiki.sugarlabs.org/api.php?action=parse&page=Activities/'+pagename+'&prop=images&format=json'

In [15]:
# getting data about images used
imagedata = requests.get(imageurl)

In [16]:
# list of all images in wiki-page

if imagedata.status_code == 200:
    
    img_json_obj = imagedata.json()
    
    imagelist = img_json_obj['parse']['images']
    
    print('List of images in wiki:')
    print(imagelist)

else:
    print('Sorry! Received status_code of GET request = {}'
          .format(imagedata.status_code))

List of images in wiki:
['CookieSearch-win.png', 'CookieSearch-lose.png', 'CookieSearch-scores.png', 'CookieSearch-icon.svg', 'CookieSearch.png', 'CookieSearch-toolbar.png']


In [17]:
# downloading each individual image

for image in imagelist:
    
    # see: https://www.mediawiki.org/wiki/API:Imageinfo
    image_info_url = 'https://wiki.sugarlabs.org/api.php?action=query&titles=File:'+image+'&prop=imageinfo&iiprop=url&format=json'
    
    # information about the image
    image_info = requests.get(image_info_url)
    
    # pages which satisfy given image condition in image_info_url
    imagepage = image_info.json()['query']['pages']
    
    #pageid for all pages in imagepage
    keys = []
    for k in imagepage:
        keys.append(k)
    
    for pageid in keys:
        
        # url of image in a given page-id
        imageurl = imagepage[pageid]['imageinfo'][0]['url']
        print(imageurl)
        
        # downloading the exact iamge
        imagefile = requests.get(imageurl)
        
        try:
            
            #imagefile_loc = pagename+'/'+image
            imagefile_loc = image
            file = open(imagefile_loc, 'wb')
            file.write(imagefile.content)
            print("File {} saved".format(image))
        finally:
            file.close()
    print()

https://wiki.sugarlabs.org/images/b/ba/CookieSearch-win.png
File CookieSearch-win.png saved

https://wiki.sugarlabs.org/images/6/62/CookieSearch-lose.png
File CookieSearch-lose.png saved

https://wiki.sugarlabs.org/images/0/09/CookieSearch-scores.png
File CookieSearch-scores.png saved

https://wiki.sugarlabs.org/images/e/ed/CookieSearch-icon.svg
File CookieSearch-icon.svg saved

https://wiki.sugarlabs.org/images/d/da/CookieSearch.png
File CookieSearch.png saved

https://wiki.sugarlabs.org/images/c/cd/CookieSearch-toolbar.png
File CookieSearch-toolbar.png saved



In [18]:
os.system('git add ./')

os.system('git commit -m "Migrate wiki to github"')

0

In [19]:
#checking remote repositories
os.system('git remote -v')

0

In [20]:
os.chdir("..")

In [21]:
#os.system('git push -u origin master')

# not working
# I'd rather push manually after verifying everything
# or use gitpython library

In [22]:
os.system('ls')

0